## 准备数据

In [61]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [62]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [ ]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([784, 256],stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([256]))
        self.W2 = tf.Variable(tf.random.normal([256, 10],stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([10]))
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        x=tf.reshape(x, [-1, 784])
        x=tf.matmul(x, self.W1) + self.b1
        x=tf.nn.relu(x)
        logits=tf.matmul(x, self.W2) + self.b2
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [64]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [ ]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.6985056 ; accuracy 0.115033336
epoch 1 : loss 2.6318803 ; accuracy 0.11726667
epoch 2 : loss 2.5738416 ; accuracy 0.12013333
epoch 3 : loss 2.5224395 ; accuracy 0.12353333
epoch 4 : loss 2.476335 ; accuracy 0.12715
epoch 5 : loss 2.4345567 ; accuracy 0.13141666
epoch 6 : loss 2.3963678 ; accuracy 0.13708334
epoch 7 : loss 2.361198 ; accuracy 0.14295
epoch 8 : loss 2.3285942 ; accuracy 0.1499
epoch 9 : loss 2.298187 ; accuracy 0.15638334
epoch 10 : loss 2.269677 ; accuracy 0.16441667
epoch 11 : loss 2.2428176 ; accuracy 0.17348333
epoch 12 : loss 2.2173984 ; accuracy 0.1822
epoch 13 : loss 2.1932447 ; accuracy 0.1908
epoch 14 : loss 2.1702065 ; accuracy 0.20036666
epoch 15 : loss 2.1481538 ; accuracy 0.21043333
epoch 16 : loss 2.126974 ; accuracy 0.22153333
epoch 17 : loss 2.1065733 ; accuracy 0.23318334
epoch 18 : loss 2.086868 ; accuracy 0.24426667
epoch 19 : loss 2.0677862 ; accuracy 0.2543
epoch 20 : loss 2.0492651 ; accuracy 0.26546666
epoch 21 : loss 2.03125 ; acc